In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [186]:
df = pd.read_csv('zno_preprocessed.csv')

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (83,84,85,86,87,88,102) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [193]:
df = df[df['year']==2020]

In [194]:
len(df.eoname.unique())

10742

In [192]:
len(df.eoname.unique())

18309

In [177]:
df =df.drop(columns=['outid']).reset_index(drop=True)

In [ ]:
ліцей податкової та рекламної справи №21 міста Києва

Ліцей №21 м. Києва

In [191]:
for i in df.eoname.unique():
    try:
        if 'Ліцей податкової та рекламної справи №21 міста Києва' in i:
            print(i)
    except:
        print('Error:',i)

Error: nan
Ліцей податкової та рекламної справи №21 міста Києва


In [23]:
from asyncio import constants
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import *


def joint_sort_descending(l1:np.array, 
                          l2:np.array)->Tuple[np.array]:
    # l1 and l2 have to be numpy arrays
    idx = np.argsort(l1)[::-1]
    return l1[idx], l2[idx]

def generate_binary_at_k(y_scores:list, 
                         k:float) -> list:
    cutoff_index = int(len(y_scores) * (k / 100.0))
    test_predictions_binary = [1 if x < cutoff_index else 0 for x in range(len(y_scores))]
    return test_predictions_binary

def metrics_at_k(y_true: list, 
                   y_scores: list, 
                   k: float) -> float:
    y_scores, y_true = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores, k)
    precision = precision_score(y_true, preds_at_k)
    recall = recall_score(y_true, preds_at_k)
    return precision,recall


class Preprocessing:
    DATA_TYPES = ['train','test']

    def __init__(self,
                 drop_strange_st_data: bool = True,
                 column_for_onehot: list = [],
                 scale_columns: list = []):

        self.drop_strange_st_data = drop_strange_st_data
        self.column_for_onehot = column_for_onehot
        self.scale_columns = scale_columns

        self.encoder = OneHotEncoder()
        self.scaler = MinMaxScaler()
        self.is_encoder_fit = False
        self.is_scaler_fit = False

    def process(self, 
                df: pd.DataFrame,
                data_type: str = 'train') -> pd.DataFrame:

        if data_type not in self.DATA_TYPES:
            raise ValueError(f'Unexpected data type. Data type must be one of {self.DATA_TYPES}.')
        
        if ((not self.is_encoder_fit) and (self.column_for_onehot)) and data_type == 'test':
            raise ValueError(f'Please, use train dataset first.')
            
        if ((not self.is_scaler_fit) and (self.scale_columns)) and data_type == 'test':
            raise ValueError(f'Please, use train dataset first.')

        if self.drop_strange_st_data:
            df = self._drop_strange_data_students_data(df)
        
        if self.column_for_onehot:
            df = self._one_hot_encoding(
                df = df,
                column_for_onehot = self.column_for_onehot,
                data_type = data_type
            )
            self.is_encoder_fit = True
        
        if self.scale_columns:
            df = self._scaling_columns(
                df = df,
                scale_columns = self.scale_columns,
                data_type = data_type
            )
            self.is_scaler_fit = True

        object_columns = self._get_object_columns(df)
        df,obj2num = self._encode_str_values(
            df = df,
            obj_columns = object_columns
        )

        self.obj2num = obj2num

        return df

    def _one_hot_encoding(self,
                          df: pd.DataFrame,
                          column_for_onehot: list,
                          data_type: str) -> pd.DataFrame:

        if data_type == 'train':
            data = self.encoder.fit_transform(df[column_for_onehot]).toarray()

        else: 
            data = self.encoder.transform(df[column_for_onehot]).toarray()

        df[self.encoder.get_feature_names_out(column_for_onehot)] = data
        df = df.drop(columns = column_for_onehot)

        return  df
    
    def _scaling_columns(self,
                         df: pd.DataFrame,
                         scale_columns: list,
                         data_type: str) -> pd.DataFrame:
        if data_type == 'train':
            df[scale_columns] = self.scaler.fit_transform(df[scale_columns])
        else: 
            df[scale_columns] = self.scaler.transform(df[scale_columns])

        return df
    
    @staticmethod
    def _encode_str_values(df: pd.DataFrame,
                           obj_columns: list) -> pd.DataFrame:
        obj2num = []
        for col in obj_columns:
            obj2num.append({col:{obj: i for i,obj in enumerate(set(df[col]))}})
            cur_num = obj2num[-1]
            df[col] = df[col].map(lambda x:cur_num[col][x])
        
        return (df,obj2num)

    @staticmethod
    def _drop_strange_data_students_data(df: pd.DataFrame) -> pd.DataFrame:
        return df.reset_index(drop=True)
    
    @staticmethod
    def _get_object_columns(df:pd.DataFrame) -> list:
        obj_col = []
        for column in df.columns:
            if df[column].dtype == 'object':
                obj_col.append(column)

        return obj_col

In [48]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report

# Regression

In [127]:
X = df[df.columns[:12]]

In [128]:
df[df.columns[25:32]]

,mathtest,mathlang,mathteststatus,mathptname,mathptregname,mathptareaname,mathpttername
0,Математика,російська,Не подолав поріг,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста
1,Математика,українська,Зараховано,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород
2,Математика,українська,Зараховано,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста
3,Математика,українська,Не з’явився,Самбірський технікум економіки та інформатики,Львівська область,м.Самбір,м.Самбір
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
379294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379295,Математика,українська,Не подолав поріг,Вище професійне училище №41 м. Тульчина,Вінницька область,Тульчинський район,м.Тульчин
379296,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379297,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
X[df.columns[26]] =  df[df.columns[26]]
X[df.columns[28:32]] =  df[df.columns[28:32]]
X

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,birth,sextypename,regname,areaname,tername,regtypename,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,mathlang,mathptname,mathptregname,mathptareaname,mathpttername
0,2003,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,Випускник української школи поточного року,"Комунальний заклад ""Маріупольська загальноосві...",середня загальноосвітня школа,Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,Департамент освіти Маріупольської міської ради...,російська,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста
1,2003,чоловіча,Полтавська область,Миргородський район,с.Хомутець,Випускник української школи поточного року,"Опорний навчальний заклад ""Хомутецька загально...",середня загальноосвітня школа,Полтавська область,Миргородський район,с.Хомутець,Відділ освіти Миргородської райдержадміністрац...,українська,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород
2,2002,чоловіча,м.Київ,м.Київ,Святошинський район міста,Випускник української школи поточного року,"Товариство з обмеженою відповідальністю ""Центр...",середня загальноосвітня школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",українська,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста
3,2004,жіноча,Львівська область,м.Львів,Галицький район міста,Студент закладу вищої освіти,Львівський коледж транспортної інфраструктури ...,заклад вищої освіти,Львівська область,м.Львів. Галицький район міста,Галицький район міста,Міністерство освіти і науки України,українська,Самбірський технікум економіки та інформатики,Львівська область,м.Самбір,м.Самбір
4,2003,жіноча,Дніпропетровська область,м.Кам'янське,Південний район міста,Студент закладу вищої освіти,Кам'янський коледж фізичного виховання,заклад вищої освіти,Дніпропетровська область,м.Кам'янське. Південний район міста,Південний район міста,Міністерство освіти і науки України,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379294,2003,чоловіча,Івано-Франківська область,Івано-Франківська область,м.Калуш,Випускник української школи поточного року,Калуська загальноосвітня школа І-ІІІ ступенів ...,середня загальноосвітня школа,Івано-Франківська область,м.Калуш,м.Калуш,Управління освіти Калуської міської ради,NaN,NaN,NaN,NaN,NaN
379295,2003,жіноча,Вінницька область,Піщанський район,с.Чорномин,Випускник української школи поточного року,"Опорний навчальний заклад ""Чорноминська загаль...",середня загальноосвітня школа,Вінницька область,Піщанський район,с.Чорномин,Відділ освіти Піщанської районної державної ад...,українська,Вище професійне училище №41 м. Тульчина,Вінницька область,Тульчинський район,м.Тульчин
379296,2003,чоловіча,Полтавська область,Полтавська область,м.Лубни,Випускник української школи поточного року,Лубенська загальноосвітня школа І-ІІІ ступенів...,середня загальноосвітня школа,Полтавська область,м.Лубни,м.Лубни,Управління освіти виконавчого комітету Лубенсь...,NaN,NaN,NaN,NaN,NaN
379297,2002,чоловіча,Вінницька область,Тульчинський район,м.Тульчин,Випускник коледжу,Вище професійне училище №41 м. Тульчина,вище професійне училище,Вінницька область,Тульчинський район,м.Тульчин,Департамент освіти і науки Вінницької обласної...,NaN,NaN,NaN,NaN,NaN


In [130]:
X['age'] = df['age'] 
X['mathball'] = df['mathball']

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [131]:
X = X.dropna().reset_index(drop=True)

In [132]:
X.head()

,birth,sextypename,regname,areaname,tername,regtypename,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,mathlang,mathptname,mathptregname,mathptareaname,mathpttername,age,mathball
0,2003,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,Випускник української школи поточного року,"Комунальний заклад ""Маріупольська загальноосві...",середня загальноосвітня школа,Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,Департамент освіти Маріупольської міської ради...,російська,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,17,8.0
1,2003,чоловіча,Полтавська область,Миргородський район,с.Хомутець,Випускник української школи поточного року,"Опорний навчальний заклад ""Хомутецька загально...",середня загальноосвітня школа,Полтавська область,Миргородський район,с.Хомутець,Відділ освіти Миргородської райдержадміністрац...,українська,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород,17,30.0
2,2002,чоловіча,м.Київ,м.Київ,Святошинський район міста,Випускник української школи поточного року,"Товариство з обмеженою відповідальністю ""Центр...",середня загальноосвітня школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",українська,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18,21.0
3,2003,жіноча,Львівська область,м.Львів,Галицький район міста,Випускник української школи поточного року,Заклад загальної середньої освіти №87 Львівськ...,середня загальноосвітня школа,Львівська область,м.Львів. Галицький район міста,Галицький район міста,Відділ освіти Галицького та Франківського райо...,українська,Середня загальноосвітня школа №84 імені Блажен...,Львівська область,м.Львів. Сихівський район міста,Сихівський район міста,17,15.0
4,2002,жіноча,Рівненська область,Сарненський район,с.Любиковичі,Випускник української школи поточного року,Любиковицька загальноосвітня школа І-ІІІ ступе...,середня загальноосвітня школа,Рівненська область,Сарненський район,с.Любиковичі,"Відділ освіти, молоді та спорту Сарненської ра...",українська,Сарненський педагогічний коледж Рівненського д...,Рівненська область,Сарненський район,м.Сарни,18,16.0


In [133]:
X.columns # add age and scale it 

Index(['birth', 'sextypename', 'regname', 'areaname', 'tername', 'regtypename',
       'eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername',
       'eoparent', 'mathlang', 'mathptname', 'mathptregname', 'mathptareaname',
       'mathpttername', 'age', 'mathball'],
      dtype='object')

In [134]:
len(X.eoparent.unique())

1408

In [135]:
train_df,test_df = train_test_split(X,test_size=0.2)

In [158]:
print(df.columns.to_list())
# 'year', 'tertypename', 'classprofilename', 'classlangname',

['birth', 'sextypename', 'regname', 'areaname', 'tername', 'regtypename', 'eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent', 'ukrtest', 'ukrteststatus', 'ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histtest', 'histlang', 'histteststatus', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathtest', 'mathlang', 'mathteststatus', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'phystest', 'physlang', 'physteststatus', 'physptname', 'physptregname', 'physptareaname', 'physpttername', 'chemtest', 'chemlang', 'chemteststatus', 'chemptname', 'chemptregname', 'chemptareaname', 'chempttername', 'biotest', 'biolang', 'bioteststatus', 'bioptname', 'bioptregname', 'bioptareaname', 'biopttername', 'geotest', 'geolang', 'geoteststatus', 'geoptname', 'geoptregname', 'geoptareaname', 'geopttername', 'engtest', 'engteststatus', 'engptname', 'engptregname', 'engptareaname', 'engpttername', 'deutest', 'deuteststatus', 'deuptna

In [167]:
df['eoname']

0         Комунальний заклад "Маріупольська загальноосві...
1         Опорний навчальний заклад "Хомутецька загально...
2         Товариство з обмеженою відповідальністю "Центр...
3         Львівський коледж транспортної інфраструктури ...
4                    Кам'янський коледж фізичного виховання
                                ...                        
379294    Калуська загальноосвітня школа І-ІІІ ступенів ...
379295    Опорний навчальний заклад "Чорноминська загаль...
379296    Лубенська загальноосвітня школа І-ІІІ ступенів...
379297              Вище професійне училище №41 м. Тульчина
379298    Гімназія "Троєщина" ІІ-ІІІ ступенів Деснянсько...
Name: eoname, Length: 379299, dtype: object

In [136]:
onehotencode_col = ["sextypename", "regname", "regtypename", "eotypename", "eoregname", "mathlang", "mathptregname"]

In [137]:
preprocessor = Preprocessing(
    drop_strange_st_data = False,
    column_for_onehot = onehotencode_col
)

In [138]:
preprocess_train_df = preprocessor.process(
    df = train_df,
    data_type = 'train'
)
preprocess_train_df

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


,birth,areaname,tername,eoname,eoareaname,eotername,eoparent,mathptname,mathptareaname,mathpttername,...,mathptregname_Рівненська область,mathptregname_Сумська область,mathptregname_Тернопільська область,mathptregname_Харківська область,mathptregname_Херсонська область,mathptregname_Хмельницька область,mathptregname_Черкаська область,mathptregname_Чернівецька область,mathptregname_Чернігівська область,mathptregname_м.Київ
93009,2002,63,4421,8134,85,4421,1279,456,170,150,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23317,2003,173,2937,9447,226,2937,655,442,64,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28184,2002,360,823,3107,455,823,488,24,97,78,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36733,2003,20,2133,7955,197,2133,700,457,31,114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31861,2003,284,3111,9172,366,3111,1366,377,86,69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91226,2003,143,1376,5424,124,1376,663,134,163,143,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
67519,2003,325,2857,5715,420,2857,603,420,89,71,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117995,2001,229,3628,3725,141,3628,961,398,189,164,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45258,2002,79,1814,4685,370,1814,961,577,65,76,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
preprocess_test_df = preprocessor.process(
    df = test_df,
    data_type = 'test'
)

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [146]:
rf_reg = RandomForestRegressor(
    random_state=42,
    max_depth=30,
    n_estimators=500
)

In [147]:
y_train = preprocess_train_df['mathball']
X_train = preprocess_train_df.drop(columns=['mathball'])
rf_reg.fit(X_train,y_train)

RandomForestRegressor(max_depth=30, n_estimators=500, random_state=42)

In [148]:
rf_reg.score(X_train,y_train)

0.5338275245691912

In [149]:
y_test = preprocess_test_df['mathball']
X_test = preprocess_test_df.drop(columns=['mathball'])
rf_reg.score(X_test,y_test)

0.20696898444667755

In [150]:
from sklearn.metrics import mean_squared_error
y_pred = rf_reg.predict(X_test)
mean_squared_error(y_test, y_pred)

146.25740731788795

In [151]:
y_pred[:5],y_test[:5]

(array([13.65417553, 19.38216221, 25.01021068, 19.06180192, 20.15600438]),
 88568    13.0
 48071     9.0
 50460     6.0
 44078    22.0
 78342    43.0
 Name: mathball, dtype: float64)

In [76]:
# regression model by general params do not work well and it is not good idea to use it. Due to Rˆ2=0.2...

# Classification

In [92]:
X = df[df.columns[:12]]
X[df.columns[26]] =  df[df.columns[26]]
X[df.columns[28:32]] =  df[df.columns[28:32]]
X['age'] = df['age'] 
X['mathteststatus'] = df['mathteststatus']

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [93]:
X.mathteststatus.unique()

array(['Не подолав поріг', 'Зараховано', 'Не з’явився', nan, 'Анульовано'],
      dtype=object)

In [94]:
X = X[(X.mathteststatus.notna()) & (X.mathteststatus!='Не з’явився')].reset_index(drop=True)
X

,birth,sextypename,regname,areaname,tername,regtypename,eoname,eotypename,eoregname,eoareaname,eotername,eoparent,mathlang,mathptname,mathptregname,mathptareaname,mathpttername,age,mathteststatus
0,2003,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,Випускник української школи поточного року,"Комунальний заклад ""Маріупольська загальноосві...",середня загальноосвітня школа,Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,Департамент освіти Маріупольської міської ради...,російська,"Комунальний заклад ""Маріупольська загальноосві...",Донецька область,м.Маріуполь. Приморський район міста,Приморський район міста,17,Не подолав поріг
1,2003,чоловіча,Полтавська область,Миргородський район,с.Хомутець,Випускник української школи поточного року,"Опорний навчальний заклад ""Хомутецька загально...",середня загальноосвітня школа,Полтавська область,Миргородський район,с.Хомутець,Відділ освіти Миргородської райдержадміністрац...,українська,Миргородська спеціалізована школа І-ІІІ ступен...,Полтавська область,м.Миргород,м.Миргород,17,Зараховано
2,2002,чоловіча,м.Київ,м.Київ,Святошинський район міста,Випускник української школи поточного року,"Товариство з обмеженою відповідальністю ""Центр...",середня загальноосвітня школа,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,"Управління освіти, молоді та спорту Святошинсь...",українська,Середня загальноосвітня школа №222 Святошинськ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18,Зараховано
3,2003,жіноча,Львівська область,м.Львів,Галицький район міста,Випускник української школи поточного року,Заклад загальної середньої освіти №87 Львівськ...,середня загальноосвітня школа,Львівська область,м.Львів. Галицький район міста,Галицький район міста,Відділ освіти Галицького та Франківського райо...,українська,Середня загальноосвітня школа №84 імені Блажен...,Львівська область,м.Львів. Сихівський район міста,Сихівський район міста,17,Зараховано
4,2002,жіноча,Рівненська область,Сарненський район,с.Любиковичі,Випускник української школи поточного року,Любиковицька загальноосвітня школа І-ІІІ ступе...,середня загальноосвітня школа,Рівненська область,Сарненський район,с.Любиковичі,"Відділ освіти, молоді та спорту Сарненської ра...",українська,Сарненський педагогічний коледж Рівненського д...,Рівненська область,Сарненський район,м.Сарни,18,Зараховано
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152076,2002,чоловіча,м.Київ,м.Київ,Святошинський район міста,Студент закладу вищої освіти,Коледж інформаційних технологій та землевпоряд...,заклад вищої освіти,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,Міністерство освіти і науки України,українська,Школа І-ІІІ ступенів № 206 імені Леся Курбаса ...,м.Київ,м.Київ. Святошинський район міста,Святошинський район міста,18,Зараховано
152077,2003,чоловіча,Чернігівська область,Чернігівська область,м.Прилуки,Випускник української школи поточного року,Прилуцький заклад загальної середньої освіти І...,ліцей,Чернігівська область,м.Прилуки,м.Прилуки,Управління освіти Прилуцької міської ради,українська,Прилуцький агротехнічний коледж,Чернігівська область,м.Прилуки,м.Прилуки,17,Зараховано
152078,2003,жіноча,Одеська область,м.Одеса,Приморський район міста,Студент закладу вищої освіти,Одеський технічний коледж Одеської національно...,заклад вищої освіти,Одеська область,м.Одеса. Приморський район міста,Приморський район міста,Міністерство освіти і науки України,українська,"Одеський навчально-виховний комплекс ""Морський...",Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,17,Зараховано
152079,2003,жіноча,Вінницька область,Піщанський район,с.Чорномин,Випускник української школи поточного року,"Опорний навчальний заклад ""Чорноминська загаль...",середня загальноосвітня школа,Вінницька область,Піщанський район,с.Чорномин,Відділ освіти Піщанської районної державної ад...,українська,Вище професійне училище №41 м. Тульчина,Ві

In [95]:
X.mathteststatus.unique()

array(['Не подолав поріг', 'Зараховано', 'Анульовано'], dtype=object)

In [96]:
def encode_statuc_binary(x):
    if x == 'Зараховано':
        return 1
    return 0

X.mathteststatus = X.mathteststatus.map(encode_statuc_binary)

In [101]:
train_df,test_df = train_test_split(X, test_size=0.2)

In [102]:
onehotencode_col = ["sextypename", "regname", "regtypename", "eotypename", "eoregname", "mathlang", "mathptregname"]
preprocessor = Preprocessing(
    drop_strange_st_data = False,
    column_for_onehot = onehotencode_col
)

In [103]:
preprocess_train_df = preprocessor.process(
    df = train_df,
    data_type = 'train'
)
preprocess_test_df = preprocessor.process(
    df = test_df,
    data_type = 'test'
)

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [121]:
rf_cf = RandomForestClassifier(
    random_state=42,
    max_depth=50,
    n_estimators=500
)

In [122]:
y_train = preprocess_train_df['mathteststatus']
X_train = preprocess_train_df.drop(columns=['mathteststatus'])
rf_cf.fit(X_train,y_train)

RandomForestClassifier(max_depth=50, n_estimators=500, random_state=42)

In [123]:
rf_cf.score(X_train,y_train)

0.9175516175697002

In [124]:
y_test = preprocess_test_df['mathteststatus']
X_test = preprocess_test_df.drop(columns=['mathteststatus'])
rf_cf.score(X_test,y_test)

0.8685932208962094

In [125]:
pred_train_y = rf_cf.predict(X_train)
pred_test_y = rf_cf.predict(X_test)

In [126]:
print('TRAIN DATA:')
print(classification_report(y_train,pred_train_y))
print('\n\n')
print('TEST DATA:')
print(classification_report(y_test,pred_test_y))

TRAIN DATA:
              precision    recall  f1-score   support

           0       0.85      0.43      0.57     15495
           1       0.92      0.99      0.95    106169

    accuracy                           0.92    121664
   macro avg       0.89      0.71      0.76    121664
weighted avg       0.91      0.92      0.91    121664




TEST DATA:
              precision    recall  f1-score   support

           0       0.39      0.06      0.11      3852
           1       0.88      0.99      0.93     26565

    accuracy                           0.87     30417
   macro avg       0.63      0.52      0.52     30417
weighted avg       0.82      0.87      0.83     30417

